# Extract UbiNet presumed degrons 

This notebook contains the code to extract each UbiNet E3-ligase presumed degrons, meaning those protein subsequences used in UbiNet for each motif construction (MSA). Specifically:
1. First, the MSAs from the HTML of each motif are parsed to extract the degrons. As UbiNet IDs are build using the structure `E3-AC6_SUBS-AC5` (AC5 meaning the accession number contains only 5 chracters), an additional matching step with the ESIs dictionary and the sequence in the proteome is required to correctly assign the AC to the substrate.
2. Second, MSA visualization to identify those alignments with indels.
3. Third, remove substrate sequences whose length differ from the motif's length (we consider these substrates are non-matrix-contributing sequences).

## Import libraries

In [1]:
import os
import sys
import pandas as pd
from bs4 import BeautifulSoup
import requests  
import gzip
import json

## my modules ##
sys.path.append("../Utils/")    # modules folder
from fasta_utils import readFasta_gzip
from motifs_utils import motif_scan

## Define variables and paths

In [1]:
base = "../../"

data = "data/"

url_motifs_path = os.path.join(base, data, "ubinet/motif_links_ubinet.txt")
ubinet_degrons_path = os.path.join(base, data, "ubinet/all_positive_set/")
ubinet_degrons_kept_path = os.path.join(base, data, "ubinet/kept_positive_set/")
ubinet_degrons_discarded_path = os.path.join(base, data, "ubinet/discarded_positive_set/")
weight_m_path = os.path.join(base, data, "ubinet/motif_matrices/PWM/")                 

## Define functions

In [3]:
# Test OK!
def map_E3_substrates(ESIs_file, scan_dir, ESIs_file_filtered = None, sep = "\t", save = True):
    """
    Generates a human-filtered E3-ligase-substrate interactions dictionary 
    
    Parameters
    ----------
    ESIs_file: str
                Path to the folder where the file containing E3-ligase-substrate interactions (ESIs) is located 
    scan_dir: str
                Path to the folder where the PWM scans are stored
    ESIs_file_filtered: str (default: None)
                Path to the folder where the filtered ESIs file will be located. Not mandatory. 
    sep: str (default: "\t")
                Column separator in the ESIs_file
    save: boolean (default: True)
                If True, the ESIs dictionary is considered to be saved. 
                If False, the ESIs dictionary is not considered to be saved. 
                By default, the function considers saving the ESIs dictionary as a file, 
                printing an error bypass message in case no ESIs_file_filtered is provided. 
                
    Returns
    -------
    ESIs: dict
                Dictionary containing E3-ligase-substrate interactions (ESIs) in the form of 
                {E3-ligase AC: [substrates AC]}
    """
    
    # Load E3-substrate interactions file
    ESIs_df = pd.read_csv(ESIs_file, sep = sep)
    
    # Generate a dictionary of E3-ligases and their corresponding substrates
    E3s = [E3.split(".")[0] for E3 in os.listdir(scan_dir)]
    ESIs = {}
    counter = 0
    for E3 in E3s:
        #print(f'E3-ligase: {E3}')
        substrates = ESIs_df.loc[(ESIs_df.E3_AC == E3) & 
                                 (ESIs_df.SUB_ORGANISM == 'Homo sapiens (Human)') & (ESIs_df.SUB_AC != '-'),
                                 "SUB_AC"].values
        substrates = list(set(substrates))     # set transformation to avoid duplicated substrates 
                                               # (problem with UbiNet data)
        ESIs[E3] = substrates
        counter += 1
    
    if save:
        try:
            with open(ESIs_file_filtered+'.json', 'w') as fp:
                json.dump(ESIs, fp)
        except TypeError:
            print("Error: name for output file not provided, saving aborted. \
            The dictionary is still generated as a variable.")
    
    print(f'Number of retrieved ESIs: {counter}')
    return ESIs
        


In [4]:
# Test OK!
def parse_motifs_msa(urlbase, links_path, ESIs, proteome, pos_set_dir):
    """
    Parses a UbiNet motif's MSA from the E3 ligase motif HTML and extracts a MSA dataframe per E3-ligase 
    containing gene, sequence, start and end columns. Replaces gene's 5-characters ACs with its corresponding 
    6-character AC
    
    Parameters
    ----------
    urlbase: str
                Non-mutable part of the URL to access the website
    links_path: str
                Path to the folder where the text file with all motifs hyperlinks will be located
    ESIs: dict
                Dictionary containing E3-ligase-substrate interactions (ESIs) in the form of {E3-ligase AC: 
                [substrates ACs]}
    proteome: dict
                Dictionary containing each protein's accession number as key and its sequence as value
    pos_set_dir: str
                Path to the folder where the parsed MSAs will be stored, named after the corresponding E3-ligase AC
    
    Returns
    -------
    None
    """
    
    counter = 0                       # (optional) Monitor progress
    

    with open(links_path) as file:
        for url in file:
            
            counter += 1              # (optional) Monitor progress
            
            E3 = url.split("/")[3]
            print(f'{counter}: {E3}')      # (optional) Monitor progress
            
            
            # Extract motifs MSA
            r = (requests.get(urlbase+url.strip())).text              
            soup = BeautifulSoup(r, 'html.parser')
            msa = str(soup.find_all(name = 'input', attrs = {"name": "aln1"}))  # string transformation required 
                                                                                # for further processing
            
            
            # MSA table processing
            msa = msa.split("*")[-1].split("\n")[1:-1]                          # splitting by *, retrieve matrix 
                                                                                # only; then, splitting by \n, 
                                                                                # separate matrix rows; indeces to 
                                                                                # keep rows only
            msa = [row.split() for row in msa]                                  # separate matrix cols
            msa_df = pd.DataFrame(msa).iloc[:, :4]                              # create df with cols: gene, 
                                                                                # sequence, start, end (first four)
            msa_df.rename(columns = {0: "gene", 
                                     1: "start", 
                                     2: "sequence", 
                                     3: "end"}, 
                          inplace = True)
            msa_df = msa_df[["gene", "sequence", "start", "end"]]               # reorder cols
            msa_df["gene"] = [gene.split("_")[1] 
                              for gene in msa_df["gene"]]                       # keep substrate 5-characters AC 
                                                                                # (remove E3-ligase AC from the 
                                                                                # name)
            msa_df["sequence"] = [seq.replace('.', '') 
                                  for seq in msa_df["sequence"]]                # remove MSA indels to obtain 
                                                                                # actual sequence
            
            
            # MSA table processing: complete gene ACs (using flag variable)
            # Check E3s independently 
            ESIs_E3 = ESIs[E3]                                                  # E3-ligase substrates ACs 
                                                                                # (according to UbiNet ESIs 
                                                                                # downloadable table)
            
            for msa_gene in msa_df["gene"]:                                     # Filter those 5-characters ACs 
                                                                                # with more than 1 match in the 
                                                                                # E3-ligase ESIs list 
                                                                                # (ambigous genes)
                # Check every E3 substrates in the ESIs dictionary
                flag = 0
                for substrate in ESIs_E3:
                    if msa_gene == substrate[:len(msa_gene)]:
                        flag += 1
                
                # When a 5-characters AC has a single match, replace it with the matching 6-characters AC
                if flag == 1:                                                       
                    for substrate in ESIs_E3:
                        if msa_gene == substrate[:len(msa_gene)]:
                            msa_df.replace({msa_gene: substrate}, inplace = True)

                # When a 5-characters AC has more than one match, identify the sequence in the proteome and 
                # replace it with the matching 6-characters AC (here, we are assuming one match is always 
                # going to happen)
                else: 
                    
                    matching_substrates = []
                    
                    # Find all posible matching substrates in ESIs dictionary
                    for substrate in ESIs_E3:
                        if msa_gene == substrate[:len(msa_gene)]:
                            matching_substrates.append(substrate)
                    #print(f'Substrates in the ESIs dictionary matching the ambigous df gene: {matching_substrates}')
                    
                    # Find all possible matching substrates in the MSA dataframe (sometimes, there is ambiguity
                    # also here)
                    ambigous_msa_df = msa_df.loc[msa_df["gene"] == msa_gene]
                    #print(f'\nDataframe subset containing the ambigous genes rows: {ambigous_msa_df}')
                    
                    # Retrieve ambigous substrates sequences
                    for index, row in ambigous_msa_df.iterrows():
                        seq, start, end = (row["sequence"], row["start"], row["end"])
                        
                        # Retrieve sequences from all possible matching substrates
                        for matching_substrate in matching_substrates:
                            #print(f'\nMatching dictionary substrate being analyzed: {matching_substrate}')

                            seq_proteome = proteome[matching_substrate][int(start)-1:int(end)]
                            #print(f'\nProteome sequence: {seq_proteome}')
                            #print(f'Dataframe sequence: {seq}')
                            
                            # Check whether the MSA sequence and the proteome-extracted sequence are the same
                            if seq == seq_proteome:
                                #print(f"{matching_substrate} Match!! Updating dataframe")
                                msa_df.update(msa_df.loc[(msa_df["start"] == start) & (msa_df["end"] == end)].
                                              replace({msa_gene: matching_substrate}))
                            #elif seq != seq_proteome:
                                #print(f"{matching_substrate} No match :(")
                            
            # Store MSA sequences (tsv file per E3-ligase)    
            #print(f'Formated MSA sequences dataframe\n {msa_df}')
            msa_df.to_csv(pos_set_dir+E3+".tsv", sep = "\t", header = True, index = False)
            
    return None

            
            
            
            
            
            

Note on the rule to find the MSA on the HTML soup: below, the HTML part where the MSA is located.

`<input type="hidden" name="aln1" value=".
                  ********
Q86YT6_Q9UHD  469 DFCIRNIE  476 + 13.3
Q86YT6_P5413 1053 DFCKKHPD 1060 + 14.5
Q86YT6_P7850  482 DHCERDID  489 + 19.9
Q86YT6_O0054  400 FNCEKKID  407 + 13.9
Q86YT6_P5335 1089 DICARDLS 1096 + 9.77
Q86YT6_Q86YT  993 PICRKAIE 1000 + 14.0
Q86YT6_Q9NYJ  348 SNCEKRVD  355 + 14.1
Q86YT6_Q9NR6  397 SNCEKKVD  404 + 16.0
Q86YT6_Q9UPN  746 QRCLRQAE  753 + 8.71
Q86YT6_O1507 2053 DHCQREQE 2060 + 12.0
Q86YT6_Q1515 1702 QRCKRKIE 1709 + 17.2
Q86YT6_Q9UNE   85 QICRRHKD   92 + 15.0
Q86YT6_Q1663  248 PICPDSLD  255 + 3.88
.">`

According to Beautiful Soup documentation, `name` corresponds to the tag at the beggining of the 'less than' sign. Attributes are whatever comes next, and MSA tables always contein `{"name": "aln1"}`.

In [12]:
def filter_motifs_msa(pos_set_dir, weight_m_dir, pos_set_filtered_dir, pos_set_discarded_dir):
    """
    Filters a dataframe of the type (gene, sequence, start, end) to only keep those sequences of the same
    length as the PWM. If applicable, generates two files of kept and discarded sequences
    
    Parameters
    ----------
    pos_set_dir: str
                Path to the folder where a dataframe of the type (gene, sequence, start, end) (positive set)
                is located
    weight_m_dir: str
                Path to the folder where the weight matrices are located. Each file has to be tab-separated
    pos_set_filtered_dir: str
                Path to the folder where the kept dataframe of the type (gene, sequence, start, end) (positive set)
                is located
    pos_set_discarded_dir: str
                Path to the folder where the discarded dataframe of the type (gene, sequence, start, end) 
                (positive set) is located
    Returns
    -------
    None
    """
    
    #counter = 0                       # (optional) Monitor progress
    
    E3s = os.listdir(pos_set_dir)
    
    for E3 in E3s:
        
        #counter += 1                   # (optional) Monitor progress
        #print(f'{counter}: {E3}')      # (optional) Monitor progress
        
        # Load MSA positive sequences and PWM of the specific E3 ligase
        msa_df = pd.read_csv(pos_set_dir+E3, sep = "\t")
        weight_m = pd.read_csv(weight_m_dir+E3, sep = "\t")
        
        # Filter the positive sequences to generate kept and discarded sequences according to PWM length
        msa_filtered_df = msa_df.loc[msa_df.sequence.apply(lambda x: len(x) == len(weight_m))]
        msa_discarded_df = msa_df.loc[msa_df.sequence.apply(lambda x: len(x) != len(weight_m))]
    
        msa_filtered_df.to_csv(pos_set_filtered_dir+E3, sep = "\t", header = True, index = False)
        if not msa_discarded_df.empty:
            msa_discarded_df.to_csv(pos_set_discarded_dir+E3, sep = "\t", header = True, index = False)
    
    return None
        
        


## 1. Load data

### 1.1. Generate ESIs dictionary

In [6]:
ESIs = map_E3_substrates(data_path+external_data_path+ESIs_file, data_path+scan_path, 
                         data_path+ESIs_file_filtered, save = False)


Number of retrieved ESIs: 104


### 1.2. Proteome loading

In [7]:
proteome = readFasta_gzip(data_path+external_data_path+proteome_file)

# Expected number of read sequences: 78120 (https://www.uniprot.org/proteomes/UP000005640)

Number of retrieved sequences: 78120


## 2. MSAs parsing

First, parse the MSAs and generate the presumed degrons dataframes.

In [8]:
urlbase = "https://awi.cuhk.edu.cn/~ubinet"

parse_motifs_msa(urlbase, url_motifs_path, ESIs, proteome, ubinet_degrons_path)

1: Q96AX9
2: Q13191
3: Q8WZ73
4: Q8TBB1
5: O76064
6: Q969Q1
7: Q13490
8: Q7Z6J0
9: Q9BYM8
10: Q8WVD3
11: Q13049
12: Q9Y4K3
13: Q13233
14: Q9Y4L5
15: Q969K3
16: Q6ZNA4
17: Q8TCQ1
18: Q86YT6
19: Q5T0T0
20: O43255
21: Q92844
22: Q96EP1
23: Q14258
24: Q9UKV5
25: Q9UHC7
26: Q86TM6
27: Q9NX47
28: Q96EQ8
29: Q00987
30: Q96F44
31: Q9NVW2
32: Q13489
33: Q96PM5
34: P22681
35: Q8TEB7
36: Q99942
37: Q8IUQ4
38: Q86YJ5
39: P78317
40: Q6VVB1
41: Q969V5
42: P98170
43: P29372
44: Q8NG27
45: Q9H4P4
46: Q96FA3
47: Q9BV68
48: O60858
49: Q9NTX7
50: P35226
51: Q99728
52: O43164
53: Q96PU5
54: O95071
55: Q96J02
56: Q9H0M0
57: Q8IYU2
58: Q9HCE7
59: Q14669
60: P46934
61: Q5T447
62: Q9HAU4
63: Q7Z6Z7
64: O00308
65: Q05086
66: Q9UNE7
67: O95155
68: O60260
69: Q9NWF9
70: Q96EP0
71: Q12834
72: Q9UM11
73: O43791
74: Q9H2C0
75: Q14145
76: Q9Y2M5
77: Q53G59
78: Q7L5Y6
79: Q9NZJ0
80: Q9Y4B6
81: Q96SW2
82: Q969H0
83: Q13309
84: Q9UK22
85: Q969P5
86: Q9Y297
87: Q9UKB1
88: Q9UKA1
89: P0C2W1
90: Q5XUX0
91: Q9UKT5
92: Q9NX

#### Motif's MSA sequences length range

After visualizing some E3 ligases MSAs, we realized sometimes they were created from sequences of different length. To avoid this, we thought of considering sequences of different length of the motif to be non-contributing to the PWM. 

First, we checked the number of E3 ligases motifs built from sequences of different length:

In [10]:
E3s = os.listdir(ubinet_degrons_path)

counter = 0                        # monitor

# Dictionary to store those E3 ligases whose positive sequences have different length
lowq_E3s = {}

for E3 in E3s:
    
    # Load MSA df
    msa_df = pd.read_csv(ubinet_degrons_path+E3, sep = "\t")
    
    # Check longest and shortest sequences
    max_len = msa_df.sequence.str.len().max()
    min_len = msa_df.sequence.str.len().min()
        
    if min_len != max_len:
        lowq_E3s[E3.split(".")[0]] = (min_len, max_len)
        print(f"{E3} motif's MSA sequences length range is {(min_len, max_len)}")
        counter += 1

print(f"\n{counter} out of {len(E3s)} E3-ligases motifs were created from very different length degrons sequences")

Q96AX9.tsv motif's MSA sequences length range is (8, 9)
Q7Z6J0.tsv motif's MSA sequences length range is (9, 11)
Q9BYM8.tsv motif's MSA sequences length range is (9, 10)
Q9Y4L5.tsv motif's MSA sequences length range is (9, 10)
O43255.tsv motif's MSA sequences length range is (7, 16)
Q92844.tsv motif's MSA sequences length range is (7, 8)
Q9UHC7.tsv motif's MSA sequences length range is (7, 8)
Q9NVW2.tsv motif's MSA sequences length range is (7, 8)
P22681.tsv motif's MSA sequences length range is (9, 10)
Q99942.tsv motif's MSA sequences length range is (8, 9)
Q8IUQ4.tsv motif's MSA sequences length range is (6, 7)
Q969V5.tsv motif's MSA sequences length range is (9, 11)
Q9NTX7.tsv motif's MSA sequences length range is (9, 10)
Q96PU5.tsv motif's MSA sequences length range is (7, 12)
Q96J02.tsv motif's MSA sequences length range is (7, 8)
O00308.tsv motif's MSA sequences length range is (8, 33)
O43791.tsv motif's MSA sequences length range is (10, 29)
Q53G59.tsv motif's MSA sequences leng

#### Alignment visualization for MSAs with different length sequences

Next, we visualize the alignments to understand what happens. Here is when we understood that the motif's length was the length of the majority of sequences, and used this rule to discard the non-contributing sequences. 

In [11]:
urlbase = "https://awi.cuhk.edu.cn/~ubinet"
counter = 0       # monitor

# Visualize MSAs from the motifs HTML of UbiNet directly
with open(url_motifs_path) as file:
    for url in file:
        if url.split("/")[3] in lowq_E3s:
            
            counter += 1
            print(f'\n{counter}: {url.split("/")[3]}')
            
            # Load corresponding PWM
            weight_m = pd.read_csv(weight_m_path+url.split("/")[3]+".tsv", sep = "\t")
            print(f"\tMotif's length: {len(weight_m.index)}")
            
            # Download corresponding MSA
            r = (requests.get(urlbase+url.strip())).text              
            soup = BeautifulSoup(r, 'html.parser')
            msa = str(soup.find_all(name = 'input', attrs = {"name": "aln1"}))
            print(msa.split('value=".')[1].split('."/>')[0])
            
            # Print MSA's sequences length range
            print(f'\tMSA sequences length range: {lowq_E3s[url.split("/")[3]]}')
            



1: Q96AX9
	Motif's length: 9

                  *********
Q96AX9_Q9NQC  105 NCEERFSLF  113 + 16.2
Q96AX9_Q9NR6  398 NCEKKVDRC  406 + 26.8
Q96AX9_Q9NYJ  349 NCEKRVDRC  357 + 26.4
Q96AX9_O0054  401 NCEKKIDYC  409 + 23.0
Q96AX9_P4798  992 NCWKKRGLC 1000 + 16.3
Q96AX9_Q9Y6K   69 N.QILRERC   76 + 3.74
Q96AX9_Q9UHD  578 N.EEQIHKF  585 + 7.43

	MSA sequences length range: (8, 9)

2: Q7Z6J0
	Motif's length: 10

                 ****.******
Q7Z6J0_Q1501 279 SILY.FYSSLS 288 + 9.34
Q7Z6J0_Q8WUM 115 SLGY.EKSCVL 124 + 11.2
Q7Z6J0_P4804 141 TIGY.GFRCVT 150 + 26.9
Q7Z6J0_O1496 693 TMGYMGSQSVS 703 + 7.30
Q7Z6J0_P6325 142 TIGY.GFRCVT 151 + 26.9
Q7Z6J0_Q9UBP 123 SIG..FRECLT 131 + 10.5

	MSA sequences length range: (9, 11)

3: Q9BYM8
	Motif's length: 10

                 **********
Q9BYM8_Q9Y6K 130 RCQQ.QMAED 138 + 13.2
Q9BYM8_Q1425  49 LCPQCRAVYQ  58 + 12.1
Q9BYM8_P1725 131 KCDTCDMNVH 140 + 15.0
Q9BYM8_P4820 119 ACPT.DLTVD 127 + 8.92
Q9BYM8_Q9BYM 446 RCPQCQIVVQ 455 + 22.5
Q9BYM8_Q8N5C 702 RCEQCEMPRY 71


                  *****..****
Q969V5_P0463  218 VPYEP..PEVG  226 + 22.0
Q969V5_P3174  447 ITITP..PDQD  455 + 8.29
Q969V5_O7538  500 RPYTPS.PQVG  509 + 14.7
Q969V5_O1535  238 VPYEP..PQVG  246 + 23.5
Q969V5_O0042  610 IPIMPASPQKG  620 + 12.2
Q969V5_Q969V  208 VRLQP..PKQG  216 + 11.2

	MSA sequences length range: (9, 11)

13: Q9NTX7
	Motif's length: 10

                  **********
Q9NTX7_P1888  447 SPQKPPTPEE  456 + 16.6
Q9NTX7_P1295  280 ALKPPPI.KL  288 + 5.72
Q9NTX7_O1516   20 APRPPVPGEE   29 + 22.0
Q9NTX7_Q9Y2T   21 APRPPVPGEE   30 + 22.0
Q9NTX7_Q9H2G  286 SVQPHSTAEL  295 + 7.70
Q9NTX7_O1523  641 ALPPPPPPHL  650 + 21.3
Q9NTX7_O9527   26 APPPPPPPPL   35 + 23.1
Q9NTX7_P0987  367 ATPPPSTASA  376 + 10.4

	MSA sequences length range: (9, 10)

14: Q96PU5
	Motif's length: 7

                  *****.....**
Q96PU5_P6274   69 LRPLS.....YP   75 + 10.8
Q96PU5_Q9NV9  171 YPVPPPY...SV  179 + 11.0
Q96PU5_P3708  191 LRVPP.....PP  197 + 14.3
Q96PU5_Q96PU  190 LPPPPL....PP  197 + 19.0
Q96PU5_Q0195   4


                  **********
Q7L5Y6_Q1308 1998 PADPANLDSE 2007 + 21.5
Q7L5Y6_P0463   87 PAPSWPLSSS   96 + 12.1
Q7L5Y6_Q1277  125 PPPPGPLSQH  134 + 17.2
Q7L5Y6_P0541  240 PPLS.PIDME  248 + 8.81
Q7L5Y6_O0076 2109 PADPANLDSE 2118 + 21.5
Q7L5Y6_Q1333  700 PPPPAPVNDE  709 + 20.6
Q7L5Y6_Q53ET  285 PPLPTPLDPE  294 + 19.9

	MSA sequences length range: (9, 10)

20: Q9Y4B6
	Motif's length: 10

                  **********
Q9Y4B6_O7544  322 EHEASR.RVK  330 + 10.1
Q9Y4B6_P1305   67 SAEQLD.RIQ   75 + 11.2
Q9Y4B6_Q7L59  160 LKERRVQRIQ  169 + 16.8
Q9Y4B6_Q9Y3Z  610 LREASKSRVQ  619 + 16.8
Q9Y4B6_P3015  411 LAEDAKWRVR  420 + 9.23
Q9Y4B6_P3524  404 EAEQEMQRIK  413 + 15.5
Q9Y4B6_O1474  650 RAERLTSRVK  659 + 15.7
Q9Y4B6_O9583  623 DEERRESRIQ  632 + 21.6
Q9Y4B6_Q9NRM  586 DEEKRESRIK  595 + 20.0

	MSA sequences length range: (9, 10)

21: Q96SW2
	Motif's length: 7

                  *******
Q96SW2_Q96SW  389 AQCKICA  395 + 16.0
Q96SW2_Q9H3D  340 FEARICA  346 + 13.9
Q96SW2_Q9UJQ  594 FQCKICG  600 + 21.6
Q96S

## 3. MSAs filtering (refinement)

After previous MSA analysis and visualization, we decided to filter the positive sequences to preserve only those contributing completely to motif generation, which are those of the same length as the motif.

In [13]:
# Maintain only MSA sequences of the same length as the motif

filter_motifs_msa(ubinet_degrons_path, weight_m_path, ubinet_degrons_kept_path, ubinet_degrons_discarded_path)